# Correlated Outputs Regularization

In [1]:
import sys
sys.path.append('..')

In [2]:
import keras_cor as kcor
import tensorflow as tf

### Simple regression NN

In [3]:
def build_mymodel(input_dim, target_corr, cor_rate=0.1, 
                  activation="sigmoid", output_dim=3):
    inputs = tf.keras.Input(shape=(input_dim,))
    h = tf.keras.layers.Dense(units=output_dim)(inputs)
    h = tf.keras.layers.Activation(activation)(h)
    outputs = kcor.CorrOutputsRegularizer(target_corr, cor_rate)(h)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

### Generate toy dataset

In [4]:
SAMPLE_SZ = 4096
INPUT_DIM = 64
OUTPUT_DIM = 4

X_train = tf.random.normal([SAMPLE_SZ, INPUT_DIM])
y_train = tf.random.normal([SAMPLE_SZ, OUTPUT_DIM])

2022-05-13 12:53:27.265365: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Cholesky trick

In [5]:
yrhos = tf.constant([
 [ 1.   , -0.884,  0.128, -0.138],
 [-0.884,  1.   , -0.555,  0.228],
 [ 0.128, -0.555,  1.   ,  0.053],
 [-0.138,  0.228,  0.053,  1.   ]
])

y_train = tf.linalg.matmul(y_train, tf.transpose(tf.linalg.cholesky(yrhos)))

### Target correlations
We need the upper triangle matrix of the correlation matrix as vector.

In [6]:
target_corr = []
for i in range(yrhos.shape[0]):
    for j in range(1 + i, yrhos.shape[1]):
        target_corr.append(yrhos[i, j])
target_corr = tf.stack(target_corr)
target_corr

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([-0.884,  0.128, -0.138, -0.555,  0.228,  0.053], dtype=float32)>

### Optimization

In [7]:
BATCH_SZ = 128
model = build_mymodel(
    input_dim=INPUT_DIM, 
    target_corr=target_corr, 
    output_dim=OUTPUT_DIM
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss="mean_squared_error"
)

history = model.fit(
    X_train, y_train, 
    batch_size=BATCH_SZ, 
    verbose=0, 
    epochs=50
)

### Inference

In [12]:
yhat = model.predict(X_train)
rhos = kcor.pearson_vec(yhat)
print(rhos)

tf.Tensor([-0.823872    0.16090639 -0.14687786 -0.522978    0.22625606  0.04918628], shape=(6,), dtype=float32)


In [11]:
print(target_corr)

tf.Tensor([-0.884  0.128 -0.138 -0.555  0.228  0.053], shape=(6,), dtype=float32)


In [14]:
print(tf.math.abs(target_corr - rhos))

tf.Tensor([0.06012797 0.03290638 0.00887786 0.032022   0.00174394 0.00381372], shape=(6,), dtype=float32)
